In [102]:
import pandas as pd
# load csv as dataframe
text_df = pd.read_csv('/Users/emilyyan/Downloads/GitHub/vandycite/gallery_buchanan/image_analysis/output/text_detection_combined.csv')
works_df = pd.read_csv('/Users/emilyyan/Downloads/GitHub/vandycite/gallery_buchanan/works_multiprop.csv')


/var/folders/jc/hmwnp8hs21qc9bfkklrxrncw0000gn/T/ipykernel_6721/2041119892.py:4: DtypeWarning: Columns (100,101,103,104,106,107,108,118,119,122,124,125,126,128,129,130,133,134) have mixed types. Specify dtype option on import or set low_memory=False.
  works_df = pd.read_csv('/Users/emilyyan/Downloads/GitHub/vandycite/gallery_buchanan/works_multiprop.csv')


In [103]:
# wrangling text detection data
text_df = text_df[['accession_number', 'locale', 'description']]
text_df = text_df.dropna(subset = ['locale'])
text_df = text_df.set_index('accession_number')

# wrangling works data
works_df = works_df[['inventory_number', 'title', 'description_en']]
works_df = works_df[works_df['description_en'].str.contains('print')]
works_df = works_df.set_index('inventory_number')


In [104]:
# fuzzy string matching function
from fuzzywuzzy import fuzz

def fuzzy_matching(first_string, second_string):
    ratio_list = []
    ratio = fuzz.ratio(first_string, second_string)
    partial_ratio = fuzz.partial_ratio(first_string, second_string)
    sort_ratio = fuzz.token_sort_ratio(first_string, second_string)
    set_ratio = fuzz.token_set_ratio(first_string, second_string)
    w_ratio = fuzz.WRatio(first_string, second_string)
    ratio_list.extend([ratio, partial_ratio, sort_ratio, set_ratio, w_ratio])
    return(ratio_list)



In [105]:
# create dataframe with accession number as index
fuzzy_ratios_df = pd.DataFrame(columns = ['accession_number', 'ratio', 'partial_ratio', 'sort_ratio', 'set_ratio', 'w_ratio'])
fuzzy_ratios_df.head()

,accession_number,ratio,partial_ratio,sort_ratio,set_ratio,w_ratio


In [107]:
for i in text_df.index:
    if i in works_df.index:
        detected_text = str(text_df.loc[i, 'description'])
        title_text = str(works_df.loc[i, 'title'])
        ratios = fuzzy_matching(detected_text, title_text)
    else:
        continue
    fuzzy_ratios_df = fuzzy_ratios_df.append({'accession_number': i, 'ratio': ratios[0], 'partial_ratio': ratios[1], 'sort_ratio': ratios[2], 'set_ratio': ratios[3], 'w_ratio': ratios[4]}, ignore_index = True)
fuzzy_ratios_df        


/var/folders/jc/hmwnp8hs21qc9bfkklrxrncw0000gn/T/ipykernel_6721/3909046826.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fuzzy_ratios_df = fuzzy_ratios_df.append({'accession_number': i, 'ratio': ratios[0], 'partial_ratio': ratios[1], 'sort_ratio': ratios[2], 'set_ratio': ratios[3], 'w_ratio': ratios[4]}, ignore_index = True)
/var/folders/jc/hmwnp8hs21qc9bfkklrxrncw0000gn/T/ipykernel_6721/3909046826.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fuzzy_ratios_df = fuzzy_ratios_df.append({'accession_number': i, 'ratio': ratios[0], 'partial_ratio': ratios[1], 'sort_ratio': ratios[2], 'set_ratio': ratios[3], 'w_ratio': ratios[4]}, ignore_index = True)
/var/folders/jc/hmwnp8hs21qc9bfkklrxrncw0000gn/T/ipykernel_6721/3909046826.py:8: FutureWarning: The frame.append method is deprecated and will be removed fr

,accession_number,ratio,partial_ratio,sort_ratio,set_ratio,w_ratio
0,1956.001,32,36,37,39,40
1,1956.002,22,35,34,48,86
2,1956.003,8,54,9,63,57
3,1956.004,19,41,26,28,86
4,1956.006,14,67,14,17,60
...,...,...,...,...,...,...
1335,2017.001.051,17,27,23,23,30
1336,2017.001.057,17,25,25,25,34
1337,2017.001.058,11,22,12,12,20
1338,2019.001,40,70,52,63,86
